In [1]:
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta
from os import makedirs
import gc
import numpy as np
import pandas as pd
from simpledbf import Dbf5
from bulletin import __file__ as __root__
from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_labels, normalize_number, normalize_municipios, normalize_igbe, normalize_hash, date_hash

pd.set_option('display.max_columns', None)
gc.enable()

In [2]:
srag = Dbf5(join('input','dbf','SRAG2020.dbf'), codec = 'cp860').to_dataframe()
srag2 = Dbf5(join('input','dbf','SRAG2021.dbf'), codec = 'cp860').to_dataframe()

In [3]:
srag = srag.append(srag2, ignore_index=True)

In [4]:
srag = srag[['NU_NOTIFIC', 'DT_NOTIFIC', 'DT_SIN_PRI', 'SG_UF_NOT', 'CO_MUN_NOT', 'ID_UNIDADE',
    'NU_CPF', 'NM_PACIENT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'CS_GESTANT',
    'CS_RACA', 'CS_ETINIA', 'NM_MAE_PAC', 'NU_CEP', 'ID_PAIS', 'SG_UF',
    'CO_MUN_RES', 'NM_BAIRRO', 'NM_LOGRADO', 'NU_NUMERO', 'FEBRE', 'TOSSE',
    'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO',
    'OUTRO_SIN', 'PUERPERA', 'CARDIOPATI', 'SIND_DOWN', 'HEPATICA', 'DIABETES',
    'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI',
    'ANTIVIRAL', 'TP_ANTIVIR', 'HOSPITAL', 'UTI', 'RAIOX_RES', 'AMOSTRA', 
    'DT_COLETA', 'REQUI_GAL', 'DT_PCR', 'TP_FLU_PCR', 'PCR_PARA1',
    'PCR_PARA2', 'PCR_PARA3', 'PCR_PARA4', 'PCR_ADENO', 'PCR_METAP', 'PCR_RESUL',
    'PCR_RINO', 'PCR_OUTRO', 'LAB_PCR', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO',
    'DT_EVOLUCA', 'DT_ENCERRA', 'NOME_PROF', 'HISTO_VGM', 'LO_PS_VGM', 'DT_VGM',
    'DT_RT_VGM', 'PCR_SARS2', 'PAC_COCBO', 'PERD_OLFT', 'PERD_PALA', 'TOMO_RES', 'NU_DO']]

In [5]:
srag = srag.rename(
    columns={
        'NU_NOTIFIC': 'id_sivep',
        'LAB_PCR': 'lab_executor',
        'DT_NOTIFIC': 'data_notificacao',
        'DT_SIN_PRI': 'data_1o_sintomas',
        'CO_MUN_NOT': 'ibge_unidade_notifica',
        'ID_UNIDADE': 'nome_unidade_notifica',
        'NU_CPF': 'cpf',
        'NM_PACIENT': 'paciente',
        'CS_SEXO': 'sexo',
        'DT_NASC': 'data_nascimento',
        'CS_RACA': 'raca_cor',
        'NM_MAE_PAC': 'nome_mae',
        'NU_CEP': 'cep_residencia',
        'CO_MUN_RES': 'ibge_residencia',
        'NM_BAIRRO': 'bairro_residencia',
        'NM_LOGRADO': 'logradouro_residencia',
        'NU_NUMERO': 'numero_residencia',
        'FEBRE': 'febre',
        'TOSSE': 'tosse',
        'GARGANTA': 'dor_garganta',
        'DISPNEIA': 'dispneia',
        'SATURACAO': 'saturacao_o2',
        'DIARREIA': 'diarreia',
        'VOMITO': 'nausea_vomitos',
        'OUTRO_SIN': 'outros_sintomas',
        'PUERPERA': 'puerperio',
        'CARDIOPATI': 'doenca_cardiovascular',
        'SIND_DOWN': 'sindrome_down',
        'HEPATICA': 'doenca_hepatica',
        'DIABETES': 'diabetes',
        'NEUROLOGIC': 'doenca_neurologica',
        'PNEUMOPATI': 'doenca_pulmonar',
        'IMUNODEPRE': 'imunodeficiencia',
        'RENAL': 'doenca_renal',
        'OBESIDADE': 'obesidade',
        'OUT_MORBI': 'outras_morbidades',
        'HOSPITAL': 'hospitalizado',
        'AMOSTRA': 'coleta_amostra',
        'DT_COLETA': 'data_coleta',
        'REQUI_GAL': 'requisicao',
        'DT_PCR': 'data_liberacao',
        'DT_EVOLUCA': 'data_cura_obito',
        'DT_ENCERRA': 'data_encerramento',
        'NOME_PROF': 'nome_notificador',
        'HISTO_VGM': 'historico_viagem',
        'LO_PS_VGM': 'local_viagem',
        'DT_VGM': 'data_ida_local',
        'DT_RT_VGM': 'data_retorno_local',
        'PERD_OLFT': 'perda_olfato_paladar',
        'NU_DO': 'numero_do',
        'PAC_COCBO': 'cod_cbo'
    })

In [6]:
srag.loc[srag['EVOLUCAO'] == '1', 'evolucao'] = '1'
srag.loc[srag['EVOLUCAO'] == '2', 'evolucao'] = '2'
srag.loc[srag['EVOLUCAO'] == '3', 'evolucao'] = '4'
srag.loc[srag['EVOLUCAO'] == '9', 'evolucao'] = '3'
srag.loc[srag['EVOLUCAO'].isnull(), 'evolucao'] = '3'

In [7]:
srag.loc[(srag['TP_IDADE'] == '1') | (srag['TP_IDADE'] == '2'), 'idade'] = 0
srag['NU_IDADE_N'] = pd.to_numeric(srag['NU_IDADE_N'], downcast = 'integer')
#srag.loc[srag['TP_IDADE'] == '3', 'idade'] = srag['NU_IDADE_N'].apply(lambda x: np.subtract(x, 400))
srag.loc[srag['TP_IDADE'] == '3', 'idade'] = srag['NU_IDADE_N']

In [8]:
srag.loc[srag['ANTIVIRAL'] == '1', 'uso_antiviral'] = '1'
srag.loc[srag['ANTIVIRAL'] == '2', 'uso_antiviral'] = '2'
srag.loc[srag['ANTIVIRAL'] == '9', 'uso_antiviral'] = ''
srag.loc[srag['TP_ANTIVIR'] == '1', 'uso_antiviral'] = '3'
srag.loc[srag['TP_ANTIVIR'] == '2', 'uso_antiviral'] = ''
srag.loc[srag['TP_ANTIVIR'] == '3', 'uso_antiviral'] = ''
srag.loc[(srag['TP_ANTIVIR'].isnull()) | (srag['ANTIVIRAL'].isnull()), 'uso_antiviral'] = ''

In [9]:
srag.loc[srag['UTI'] == '1', 'tipo_internacao'] = '2'
srag.loc[((srag['UTI'] == '2') | (srag['UTI'] == '9') | (srag['UTI'].isnull())) & (srag['hospitalizado'] == '1'), 'tipo_internacao'] = '3'
srag.loc[srag['hospitalizado'] != '1', 'tipo_internacao'] = '3'

In [10]:
srag.loc[srag['RAIOX_RES'] == '1', 'raiox_torax'] = '1'
srag.loc[srag['RAIOX_RES'] == '4', 'raiox_torax'] = '2'
srag.loc[srag['RAIOX_RES'] == '2', 'raiox_torax'] = '3'
srag.loc[srag['RAIOX_RES'] == '3', 'raiox_torax'] = '4'
srag.loc[srag['RAIOX_RES'] == '5', 'raiox_torax'] = '5'
srag.loc[(srag['RAIOX_RES'].isnull()) | (srag['RAIOX_RES'] == '6') | (srag['RAIOX_RES'] == '9'), 'raiox_torax'] = ''

In [11]:
srag.loc[srag['CLASSI_FIN'] == '1', 'classificacao_final'] = '3'
srag.loc[srag['CLASSI_FIN'] == '2', 'classificacao_final'] = '3'
srag.loc[srag['CLASSI_FIN'] == '3', 'classificacao_final'] = '3'
srag.loc[srag['CLASSI_FIN'] == '4', 'classificacao_final'] = '5'
srag.loc[srag['CLASSI_FIN'] == '5', 'classificacao_final'] = '2'
srag.loc[srag['CLASSI_FIN'].isnull(), 'classificacao_final'] = '1'

In [12]:
srag.loc[srag['CRITERIO'] == '1', 'criterio_classificacao'] = '1'
srag.loc[srag['CRITERIO'] == '2', 'criterio_classificacao'] = '2'
srag.loc[srag['CRITERIO'] == '3', 'criterio_classificacao'] = '6'
srag.loc[srag['CRITERIO'] == '4', 'criterio_classificacao'] = '5'
srag.loc[srag['CRITERIO'].isnull(), 'criterio_classificacao'] = '3'

In [13]:
srag.loc[srag['TOMO_RES'] == 1, 'tomografia'] = '1'
srag.loc[((srag['TOMO_RES'] == 2) | (srag['TOMO_RES'] == 3) |
         (srag['TOMO_RES'] == 4) | (srag['TOMO_RES'] == 5)), 'tomografia'] = '4'
srag.loc[((srag['TOMO_RES'] == 6) | (srag['TOMO_RES'] == 9) | (srag['TOMO_RES'] == 0)), 'tomografia'] = ''

In [14]:
srag.loc[srag['CS_GESTANT'] == '0', 'gestante'] = '3'
srag.loc[(srag['CS_GESTANT'] == '1') | (srag['CS_GESTANT'] == '2') | (srag['CS_GESTANT'] == '3') |
         (srag['CS_GESTANT'] == '4'), 'gestante'] = '1'
srag.loc[srag['CS_GESTANT'] == '5', 'gestante'] = '2'
srag.loc[(srag['CS_GESTANT'] == '6') | (srag['CS_GESTANT'] == '9'), 'gestante'] = '3'

In [15]:
srag.loc[srag['CS_GESTANT'] == '1', 'periodo_gestacao'] = '1'
srag.loc[srag['CS_GESTANT'] == '2', 'periodo_gestacao'] = '2'
srag.loc[srag['CS_GESTANT'] == '3', 'periodo_gestacao'] = '3'
srag.loc[srag['CS_GESTANT'] == '4', 'periodo_gestacao'] = '4'
srag.loc[(srag['gestante'] == '2') | (srag['gestante'] == '3'), 'periodo_gestacao'] = ""

In [16]:
srag.loc[srag['PCR_RESUL'] == '1', 'resultado'] = '12'
srag.loc[srag['PCR_RESUL'] == '2', 'resultado'] = '13'
srag.loc[srag['PCR_RESUL'] == '3', 'resultado'] = '14'
srag.loc[((srag['PCR_RESUL'] == '4') | (srag['PCR_RESUL'] == '5') | (srag['PCR_RESUL'] == '9') |
        (srag['PCR_RESUL'].isnull())), 'resultado'] = ''
srag.loc[srag['TP_FLU_PCR'] == '1', 'resultado'] = '1'
srag.loc[srag['TP_FLU_PCR'] == '2', 'resultado'] = '2'
srag.loc[srag['PCR_PARA1'] == '1', 'resultado'] = '5'
srag.loc[srag['PCR_PARA2'] == '1', 'resultado'] = '5'
srag.loc[srag['PCR_PARA3'] == '1', 'resultado'] = '5'
srag.loc[srag['PCR_PARA4'] == '1', 'resultado'] = '5'
srag.loc[srag['PCR_ADENO'] == '1', 'resultado'] = '3'
srag.loc[srag['PCR_METAP'] == '1', 'resultado'] = '7'
srag.loc[srag['PCR_RINO'] == '1', 'resultado'] = '6'
srag.loc[srag['PCR_OUTRO'] == '1', 'resultado'] = '10'
srag.loc[srag['PCR_SARS2'] == '1', 'resultado'] = '9'

In [17]:
srag.loc[srag['raca_cor'] == '3', 'raca_cor'] = '-1'
srag.loc[srag['raca_cor'] == '4', 'raca_cor'] = '3'
srag.loc[srag['raca_cor'] == '-1', 'raca_cor'] = '4'
srag.loc[srag['raca_cor'] == '9', 'raca_cor'] = '99'
srag.loc[srag['raca_cor'].isnull(), 'raca_cor'] = '99'

In [18]:
srag.loc[srag['febre'] == '1', 'febre'] = '1'
srag.loc[srag['febre'] == '2', 'febre'] = '2'
srag.loc[srag['febre'] == '9', 'febre'] = '3'
srag.loc[srag['febre'].isnull(), 'febre'] = '3'

In [19]:
srag.loc[srag['tosse'] == '1', 'tosse'] = '1'
srag.loc[srag['tosse'] == '2', 'tosse'] = '2'
srag.loc[srag['tosse'] == '9', 'tosse'] = '3'
srag.loc[srag['tosse'].isnull(), 'tosse'] = '3'

In [20]:
srag.loc[srag['dor_garganta'] == '1', 'dor_garganta'] = '1'
srag.loc[srag['dor_garganta'] == '2','dor_garganta'] = '2'
srag.loc[srag['dor_garganta'] == '9', 'dor_garganta'] = '3'
srag.loc[srag['dor_garganta'].isnull(), 'dor_garganta'] = '3'

In [21]:
srag.loc[(srag['dispneia'] == '1') | (srag['DESC_RESP'] == '1'), 'dispneia'] = '1'
srag.loc[(srag['dispneia'] == '2') | (srag['DESC_RESP'] == '2'), 'dispneia'] = '2'
srag.loc[(srag['dispneia'] == '9') | (srag['DESC_RESP'] == '9'), 'dispenia'] = '3'
srag.loc[(srag['dispneia'].isnull()) & (srag['DESC_RESP'].isnull()), 'dispenia'] = '3'

In [22]:
srag.loc[srag['saturacao_o2'] == '1', 'saturacao_o2'] = '1'
srag.loc[srag['saturacao_o2'] == '2','saturacao_o2'] = '2'
srag.loc[srag['saturacao_o2'] == '9', 'saturacao_o2'] = '3'
srag.loc[srag['saturacao_o2'].isnull(), 'saturacao_o2'] = '3'

In [23]:
srag.loc[srag['diarreia'] == '1', 'diarreia'] = '1'
srag.loc[srag['diarreia'] == '2','diarreia'] = '2'
srag.loc[srag['diarreia'] == '9', 'diarreia'] = '3'
srag.loc[srag['diarreia'].isnull(), 'diarreia'] = '3'

In [24]:
srag.loc[srag['nausea_vomitos'] == '1', 'nausea_vomitos'] = '1'
srag.loc[srag['nausea_vomitos'] == '2','nausea_vomitos'] = '2'
srag.loc[srag['nausea_vomitos'] == '9', 'nausea_vomitos'] = '3'
srag.loc[srag['nausea_vomitos'].isnull(), 'nausea_vomitos'] = '3'

In [25]:
srag.loc[srag['puerperio'] == '1', 'puerperio'] = '1'
srag.loc[srag['puerperio'] == '2','puerperio'] = '2'
srag.loc[srag['puerperio'] == '9', 'puerperio'] = '3'
srag.loc[srag['puerperio'].isnull(), 'puerperio'] = '3'

In [26]:
srag.loc[srag['doenca_cardiovascular'] == '1', 'doenca_cardiovascular'] = '1'
srag.loc[srag['doenca_cardiovascular'] == '2','doenca_cardiovascular'] = '2'
srag.loc[srag['doenca_cardiovascular'] == '9', 'doenca_cardiovascular'] = '3'
srag.loc[srag['doenca_cardiovascular'].isnull(), 'doenca_cardiovascular'] = '3'

In [27]:
srag.loc[srag['doenca_hepatica'] == '1', 'doenca_hepatica'] = '1'
srag.loc[srag['doenca_hepatica'] == '2','doenca_hepatica'] = '2'
srag.loc[srag['doenca_hepatica'] == '9', 'doenca_hepatica'] = '3'
srag.loc[srag['doenca_hepatica'].isnull(), 'doenca_hepatica'] = '3'

In [28]:
srag.loc[srag['diabetes'] == '1', 'diabetes'] = '1'
srag.loc[srag['diabetes'] == '2','diabetes'] = '2'
srag.loc[srag['diabetes'] == '9', 'diabetes'] = '3'
srag.loc[srag['diabetes'].isnull(), 'diabetes'] = '3'

In [29]:
srag.loc[srag['doenca_neurologica'] == '1', 'doenca_neurologica'] = '1'
srag.loc[srag['doenca_neurologica'] == '2','doenca_neurologica'] = '2'
srag.loc[srag['doenca_neurologica'] == '9', 'doenca_neurologica'] = '3'
srag.loc[srag['doenca_neurologica'].isnull(), 'doenca_neurologica'] = '3'

In [30]:
srag.loc[srag['doenca_pulmonar'] == '1', 'doenca_pulmonar'] = '1'
srag.loc[srag['doenca_pulmonar'] == '2','doenca_pulmonar'] = '2'
srag.loc[srag['doenca_pulmonar'] == '9', 'doenca_pulmonar'] = '3'
srag.loc[srag['doenca_pulmonar'].isnull(), 'doenca_pulmonar'] = '3'

In [31]:
srag.loc[srag['doenca_renal'] == '1', 'doenca_renal'] = '1'
srag.loc[srag['doenca_renal'] == '2','doenca_renal'] = '2'
srag.loc[srag['doenca_renal'] == '9', 'doenca_renal'] = '3'
srag.loc[srag['doenca_renal'].isnull(), 'doenca_renal'] = '3'

In [32]:
srag.loc[srag['obesidade'] == '1', 'obesidade'] = '1'
srag.loc[srag['obesidade'] == '2','obesidade'] = '2'
srag.loc[srag['obesidade'] == '9', 'obesidade'] = '3'
srag.loc[srag['obesidade'].isnull(), 'obesidade'] = '3'

In [33]:
srag.loc[srag['hospitalizado'] == '1', 'hospitalizado'] = '1'
srag.loc[srag['hospitalizado'] == '2','hospitalizado'] = '2'
srag.loc[srag['hospitalizado'] == '9', 'hospitalizado'] = '3'
srag.loc[srag['hospitalizado'].isnull(), 'hospitalizado'] = '3'

In [34]:
srag.loc[srag['coleta_amostra'] == '1', 'coleta_amostra'] = '1'
srag.loc[srag['coleta_amostra'] == '2','coleta_amostra'] = '2'
srag.loc[srag['coleta_amostra'] == '9', 'coleta_amostra'] = '3'
srag.loc[srag['coleta_amostra'].isnull(), 'coleta_amostra'] = '3'

In [35]:
srag.loc[srag['historico_viagem'] == '1', 'historico_viagem'] = '1'
srag.loc[srag['historico_viagem'] == '2','historico_viagem'] = '2'
srag.loc[srag['historico_viagem'].isnull(), 'historico_viagem'] = ''

In [36]:
srag.loc[(srag['perda_olfato_paladar'] == '1') | (srag['PERD_PALA'] == '1'), 'perda_olfato_paladar'] = '1'
srag.loc[(srag['perda_olfato_paladar'] == '2') | (srag['PERD_PALA'] == '2'), 'perda_olfato_paladar'] = '2'
srag.loc[(srag['perda_olfato_paladar'] == '9') | (srag['PERD_PALA'] == '9'), 'perda_olfato_paladar'] = '3'
srag.loc[(srag['perda_olfato_paladar'].isnull()) & (srag['PERD_PALA'].isnull()), 'perda_olfato_paladar'] = '3'

In [37]:
srag.loc[srag['sexo'] == 'M', 'sexo'] = '1'
srag.loc[srag['sexo'] == 'F','sexo'] = '2'
srag.loc[srag['sexo'] == 'I','sexo'] = '2'

In [38]:
srag["data_notificacao"] = srag["data_notificacao"].apply(pd.to_datetime)
srag["data_1o_sintomas"] = srag["data_1o_sintomas"].apply(pd.to_datetime)
srag["data_nascimento"] = srag["data_nascimento"].apply(pd.to_datetime)
srag["data_liberacao"] = srag["data_liberacao"].apply(pd.to_datetime)
srag["data_cura_obito"] = srag["data_cura_obito"].apply(pd.to_datetime)
srag["data_encerramento"] = srag["data_encerramento"].apply(pd.to_datetime)
srag["data_ida_local"] = srag["data_ida_local"].apply(pd.to_datetime)
srag["data_retorno_local"] = srag["data_retorno_local"].apply(pd.to_datetime)

In [39]:
municipios = static.municipios[['cod_uf','ibge']]
municipios = municipios.rename(columns={'ibge':'ibge_residencia', 'cod_uf': 'uf_residencia'})
municipios['uf_residencia'] = municipios['uf_residencia'].astype('string')
srag['ibge_residencia'] = pd.to_numeric(srag['ibge_residencia'], downcast = 'integer')
srag = pd.merge(left=srag, right=municipios, how='left', on='ibge_residencia')
srag.loc[srag['uf_residencia'].isnull(), 'uf_residencia'] = '0'

In [40]:
municipios = municipios.rename(columns={'ibge_residencia':'ibge_unidade_notifica', 'uf_residencia': 'uf_unidade_notifica'})
municipios['uf_unidade_notifica'] = municipios['uf_unidade_notifica'].astype('string')
srag['ibge_unidade_notifica'] = pd.to_numeric(srag['ibge_unidade_notifica'], downcast = 'integer')
srag = pd.merge(left=srag, right=municipios, how='left', on='ibge_unidade_notifica')
srag.loc[srag['uf_unidade_notifica'].isnull(), 'uf_unidade_notifica'] = '0'

In [41]:
pais = static.pais[['co_pais','ds_pais']]
pais = pais.rename(columns={'ds_pais':'ID_PAIS', 'co_pais': 'pais_residencia'})
pais['pais_residencia'] = pais['pais_residencia'].astype('string')
srag['ID_PAIS'] = pais['ID_PAIS'].astype('string')
srag = pd.merge(left=srag, right=pais, how='left', on='ID_PAIS')
srag.loc[srag['pais_residencia'].isnull(), 'pais_residencia'] = '0'

In [42]:
etnia = static.etnia[['co_etnia','etnia']]
etnia = etnia.rename(columns={'etnia':'CS_ETINIA', 'co_etnia': 'etnia'})
etnia['etnia'] = etnia['etnia'].astype('string')
srag['CS_ETINIA'] = srag['CS_ETINIA'].astype('string')
srag = pd.merge(left=srag, right=etnia, how='left', on='CS_ETINIA')
srag.loc[srag['etnia'].isnull(), 'etnia'] = '0'

In [43]:
srag.loc[srag['nome_mae'].isnull(), 'nome_mae'] = ''
#Conjunto de negações encontradas na coluna nome_mae
nao = set(['NAO','CONSTA','INFO','INFORMADO','CONTEM',''])
#Anula campo com alguma negação na coluna nome_mae
srag.loc[ [True if set(nome_mae.split(" ")).intersection(nao) else False for nome_mae in srag['nome_mae'] ], 'nome_mae'] = None

In [44]:
srag = srag[['data_notificacao', 'data_1o_sintomas', 'uf_unidade_notifica', 'ibge_unidade_notifica', 'nome_unidade_notifica',
             'cpf', 'paciente', 'sexo', 'data_nascimento', 'idade', 'gestante', 'periodo_gestacao', 'raca_cor', 'etnia',
             'nome_mae', 'cep_residencia', 'pais_residencia', 'uf_residencia', 'ibge_residencia', 'bairro_residencia',
             'logradouro_residencia', 'numero_residencia', 'febre', 'tosse', 'dor_garganta', 'dispneia', 'dispneia',
             'saturacao_o2', 'diarreia', 'nausea_vomitos', 'outros_sintomas', 'puerperio', 'doenca_cardiovascular',
             'sindrome_down', 'doenca_hepatica', 'diabetes', 'doenca_neurologica', 'doenca_pulmonar', 'imunodeficiencia',
             'doenca_renal', 'obesidade', 'outras_morbidades', 'uso_antiviral', 'hospitalizado', 'tipo_internacao',
             'raiox_torax', 'coleta_amostra', 'data_coleta', 'requisicao', 'data_liberacao', 'resultado', 'lab_executor',
             'classificacao_final', 'criterio_classificacao', 'evolucao', 'data_cura_obito', 'data_encerramento',
             'nome_notificador', 'historico_viagem', 'local_viagem', 'data_ida_local', 'data_retorno_local', 'id_sivep',
             'perda_olfato_paladar', 'perda_olfato_paladar', 'tomografia', 'numero_do', 'cod_cbo']]

In [45]:
srag

,data_notificacao,data_1o_sintomas,uf_unidade_notifica,ibge_unidade_notifica,nome_unidade_notifica,cpf,paciente,sexo,data_nascimento,idade,gestante,periodo_gestacao,raca_cor,etnia,nome_mae,cep_residencia,pais_residencia,uf_residencia,ibge_residencia,bairro_residencia,logradouro_residencia,numero_residencia,febre,tosse,dor_garganta,dispneia,dispneia,saturacao_o2,diarreia,nausea_vomitos,outros_sintomas,puerperio,doenca_cardiovascular,sindrome_down,doenca_hepatica,diabetes,doenca_neurologica,doenca_pulmonar,imunodeficiencia,doenca_renal,obesidade,outras_morbidades,uso_antiviral,hospitalizado,tipo_internacao,raiox_torax,coleta_amostra,data_coleta,requisicao,data_liberacao,resultado,lab_executor,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,data_encerramento,nome_notificador,historico_viagem,local_viagem,data_ida_local,data_retorno_local,id_sivep,perda_olfato_paladar,perda_olfato_paladar,tomografia,numero_do,cod_cbo
0,2019-12-31,2019-12-30,41,410830,HOSPITAL MUNICIPAL PADRE GERMANO LAUCK,NaN,DAVI VIEIRA WERK,1,2017-08-22,2.0,3,,2,0,JENIFFER DA SILVA VIEIRA,NaN,1,41,410830.0,OURO VERDE,RUA NIQUEL,480,1,1,2,1,1,2,2,2,2,3,3,NaN,3,3,3,3,NaN,3,3,NaN,,1,3,1,1,31/12/2019,200903000009,2020-01-15,7,LABORATORIO CENTRAL DO ESTADO,3,1,1,2020-02-01,2020-01-20,PRISCILLA GOMES GUERRA,0,NaN,NaT,NaT,315779611693,3,3,,0.0,NaN
1,2020-02-01,2019-12-29,41,410830,HOSPITAL MUNICIPAL PADRE GERMANO LAUCK,NaN,MARIA CLARA MEYER CARNEIRO,2,2019-07-22,0.0,3,,1,0,DARY ENES CRUSTINA MEYER MARTINS,NaN,2,41,410830.0,JD LANCASTER,RUA BIGAUCU,755,1,1,2,1,1,2,2,1,2,3,3,NaN,3,3,3,3,NaN,3,3,NaN,,1,3,1,1,02/01/2020,200903000010,2020-01-15,7,LABORATORIO CENTRAL DO ESTADO,3,1,1,2020-05-01,2020-01-20,DEJAMIR,0,NaN,NaT,NaT,315779617056,3,3,,0.0,NaN
2,2020-03-01,2019-12-30,41,411370,ISCAL,13647278890,MARINA CERQUEIRA CESAR,2,1926-11-08,93.0,2,,99,0,ALINA CORREA CAMARGO,19900021,3,35,353470.0,CENTRO,PARANA 570/1171,830,1,1,2,1,1,2,2,2,2,3,3,NaN,3,3,3,3,NaN,3,3,NaN,3,1,3,3,1,02/01/2020,201703000034,2020-01-15,13,NaN,5,1,1,2020-05-01,2020-01-16,JHESSICA ALVES,0,NaN,NaT,NaT,315780600456,3,3,,0.0,NaN
3,2020-03-01,2020-01-01,41,411330,CENTRO MEDICO HOSPITALAR SAO LUCAS,NaN,DIEGO GUILHMERME DA SILVA EINSSELD,1,2018-09-29,1.0,3,,3,0,KEZIA DA SILVA,NaN,4,41,411705.0,INTERIOR,GUAJUVIRA,SN,1,1,2,1,1,1,2,2,2,3,3,NaN,3,3,3,3,NaN,3,3,NaN,3,1,2,3,1,03/01/2020,200501000001,2020-01-15,5,NaN,3,1,1,NaT,2020-01-15,PATRICIA MASSUQUETO,0,NaN,NaT,NaT,315780619266,3,3,,0.0,NaN
4,2020-02-01,2020-01-01,41,410370,SANTA CASA DE CAMBE,NaN,BRUNO RAFAEL DE OLIVEIRA,1,1988-12-08,31.0,3,,1,0,MARLENE PEREIRA DE SOUZA,NaN,5,41,410370.0,CONJUNTO CRISTAL,RUA BRILHANTE,213,1,1,3,1,1,3,1,1,9,2,2,2,2,2,2,2,1,2,2,2,3,1,2,4,1,02/01/2020,201710000008,2020-01-15,13,NaN,5,1,4,2020-03-01,2020-01-21,NaN,0,NaN,NaT,NaT,315780756981,3,3,,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86470,2021-02-19,2021-08-02,41,411370,HOSPITAL UNIVERSITARIO REGIONAL DO NORTE DO PA...,03365933921,SYLVIA KARINE DE DEUS BUSSULO,2,1981-03-08,39.0,2,,1,0,ILIACY APARECIDA CAPACCI DE DEUS,86200000,0,41,410980.0,NaN,RUA AMETISTA,120,2,2,2,2,2,2,2,2,2,3,3,NaN,3,3,3,3,NaN,3,3,NaN,,2,3,,2,NaN,NaN,NaT,,NaN,1,3,3,NaT,NaT,ISADORA,2,NaN,NaT,NaT,316137582189,2,2,4,0.0,NaN
86471,2021-02-19,2021-02-17,41,411520,SANTA CASA DE MARINGA HOSPITAL E MAT M AUXILIA...,12144224978,LUAN EMANOEL SANTOS SIQUEIRA,1,2014-05-12,6.0,3,,1,0,EDINA DE FATIMA DOS SANTOS,86970000,0,41,410655.0,CENTRO,RUA ITAPOA,132,2,2,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,,1,3,3,1,17/02/2021,NaN,NaT,,NaN,1,3,3,NaT,NaT,NaN,2,NaN,NaT,NaT,316137582817,2,2,,0.0,NaN
86472,2021-02-17,2021-07-02,41,410690,ONIX MATEUS LEME,06748292980,JEAN LUIZ BENEVENUTI,1,1982-01-02,39.0,3,,99,0,ELIETE ANA SANTANA,82640350,0,41,410690.0,SANTA CANDIDA,DAMASINA BI